**Dictionary for MIRA users**



In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
from google.colab import files

In [ ]:
user_info = pd.read_csv('/content/hub_user_info_export_2024-02-07_110056.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/hub_user_info_export_2024-02-07_110056.csv'

In [ ]:
df = pd.read_csv('/content/full_dataset_one_row_per_cycle_fw_indexing_48days.csv')

In [ ]:
cycle_data = pd.read_csv('/content/hub_user_cycle_export_2024-02-07_095939.csv')

In [ ]:
user_info.head()

,id,hub_id,birth_year,birth_month,goal_status,conditions
0,1,U2921516,1986,1.0,2,3
1,2,U2ABB1615570212,1981,10.0,2,1
2,3,U2ABC3515570773,1985,3.0,2,"1,3"
3,4,U2ABEA415572434,1991,12.0,2,"0,2"
4,5,U2AD07F15606021,1984,NaN,0,0


In [ ]:
with_lh_spike = pd.merge(user_info, df, on='hub_id')

In [ ]:
user_info['birth_month'].fillna(1, inplace=True)

In [ ]:
user_info.dropna(subset=['birth_year'], inplace=True)

In [ ]:
cycle_data['date_period_start'] = pd.to_datetime(cycle_data['date_period_start'])
cycle_data['period_year'] = cycle_data['date_period_start'].dt.year
cycle_data['period_month'] = cycle_data['date_period_start'].dt.month

In [ ]:
def age_months(birth_year, birth_month, period_year, period_month):
    age_years = period_year - birth_year
    age_months = period_month - birth_month
    if age_months < 0:
        age_years -= 1
        age_months += 12
    total_age_months = (age_years * 12) + age_months
    return total_age_months

In [ ]:
data = []

# Iterate over user info
for index, row in user_info.iterrows():
    hub_id = row['hub_id']
    birth_year = int(row['birth_year'])
    birth_month = int(row['birth_month'])

    # Get cycle data for the specific hub_id
    per_id = cycle_data[cycle_data['hub_id'] == hub_id]

    # Initialize cycle_index for this user
    cycle_index = 0

    # Iterate over each cycle for the user
    for _, cycle_row in per_id.iterrows():
        period_year = cycle_row['period_year']
        period_month = cycle_row['period_month']

        # Calculate age in months at the time of this cycle
        age_in_months = age_months(birth_year, birth_month, period_year, period_month)

        date_of_birth = f"{int(row['birth_month']):02d}/{int(row['birth_year'])}"

        # Append row to data list with hub_id, cycle_index, and age information
        data.append({
            'hub_id': hub_id,
            'cycle_index': cycle_index,
            'date_of_birth': date_of_birth,
            'age_in_months': age_in_months
        })

        # Increment cycle_index for the next cycle entry for this user
        cycle_index += 1

In [ ]:
df = pd.DataFrame(data)
df.to_csv('user_dictionary_per_user_cycle.csv', index=False)
files.download('user_dictionary_per_user_cycle.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>